<span>
<img src="http://ndlib.readthedocs.io/en/latest/_static/ndlogo2.png" width="240px" align="right"/>
</span>
<span>
<b>Author:</b> <a href="http://about.giuliorossetti.net">Giulio Rossetti</a><br/>
<b>Python version:</b>  3.6<br/>
<b>NDlib version:</b>  4.0.1<br/>
<b>Last update:</b> 25/09/2018
</span>

<a id='top'></a>
# *Intro to NDlib-REST: Remote experiment definition and execution*

[``NDlib``](http://ndlib.readthedocs.io) is a python library designed to provide support to analysis of diffusive phenomena occurring on top of complex network structures.

[``NDlib-REST``](http://ndlib.readthedocs.io/en/latest/rest/ndlib-rest.html) is a restful server that allows to decouple experiment definition (client side) and execution (server side). 

In this notebook is introduced the python endpoint workflow to setup a diffusion experiment using the [``NDlib-REST``](http://ndlib.readthedocs.io/en/latest/rest/ndlib-rest.html) API.

**Note:** this notebook is purposely not 100% comprehensive, it only discusses the basic things you need to get started.

## Table of Contents

1. [Setup the Experiment Server](#install)
2. [Experiment definition and execution](#workflow)

<a id='install'></a>
## 1. Setup the Experiment Server ([to top](#top))

**1)** Move to the working directory

In [7]:
!rm -rf ndlib-rest

**2)** Clone the ``NDlib-REST`` server project from the project [GitHub](https://github.com/GiulioRossetti/ndlib-rest) 

In [8]:
!git clone https://github.com/GiulioRossetti/ndlib-rest.git > /dev/null

Cloning into 'ndlib-rest'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1319 (delta 0), reused 0 (delta 0), pack-reused 1316 
Ricezione degli oggetti: 100% (1319/1319), 8.20 MiB | 504.00 KiB/s, done.
Risoluzione dei delta: 100% (752/752), done.


**3)** Start the REST server

In [9]:
%cd ndlib-rest/
import subprocess as sub
sub.call('./gunicorn.sh', shell=True)

/home/rossetti/git/CNA_Tutorials/ndlib-rest/ndlib-rest


0

<a id='workflow'></a>
## 2. Client for remote experiment setup ([to top](#top))

``NDlibClient``, installed along with the REST server, provides a python wrapper around the endpont calls offered by ``NDlib-REST``. <br/>

For endpoint parameters specs refer to the <a href="/docs">Service API</a>.

In [10]:
from client.NDlibClient import NDlibClient

**1)** Specify the base url of the server exposing the rest API

In [11]:
exp = NDlibClient("http://127.0.0.1:5000")

**2)** Create an experiment

In [12]:
exp.create_experiment()

**3)** Add a graph resource to the experiment

In [13]:
resp = exp.add_erdos_renyi_graph(1000, 0.1)

**4)** Add one or more diffusion models to the experiment

In [14]:
resp = exp.add_SIR(infected=0.01, beta=0.001, gamma=0.09)

**5)** Execute the simulation

In [15]:
it = exp.get_iteration_bunch(bunch=15)

**6)** Analyze the results

In [16]:
models = it.keys()

In [17]:
for m in models:
    print("Model: %s\n" % m)
    for i in range(0, len(it[m])):
        print("Iteration: %s\t Infected: %s\t Susceptible: %s\t Removed: %s" % (it[m][i]['iteration'], 
         it[m][i]['node_count']['1'], 
         it[m][i]['node_count']['0'], 
         it[m][i]['node_count']['2']))

Model: SIR_0

Iteration: 0	 Infected: 10	 Susceptible: 990	 Removed: 0
Iteration: 1	 Infected: 7	 Susceptible: 990	 Removed: 3
Iteration: 2	 Infected: 7	 Susceptible: 990	 Removed: 3
Iteration: 3	 Infected: 6	 Susceptible: 990	 Removed: 4
Iteration: 4	 Infected: 7	 Susceptible: 989	 Removed: 4
Iteration: 5	 Infected: 8	 Susceptible: 988	 Removed: 4
Iteration: 6	 Infected: 8	 Susceptible: 987	 Removed: 5
Iteration: 7	 Infected: 8	 Susceptible: 987	 Removed: 5
Iteration: 8	 Infected: 9	 Susceptible: 984	 Removed: 7
Iteration: 9	 Infected: 10	 Susceptible: 982	 Removed: 8
Iteration: 10	 Infected: 13	 Susceptible: 979	 Removed: 8
Iteration: 11	 Infected: 11	 Susceptible: 979	 Removed: 10
Iteration: 12	 Infected: 13	 Susceptible: 975	 Removed: 12
Iteration: 13	 Infected: 12	 Susceptible: 974	 Removed: 14
Iteration: 14	 Infected: 12	 Susceptible: 972	 Removed: 16


**7)** Destroy the experiment and free the assigned resources

In [18]:
resp = exp.destroy_experiment()